In [1]:
import cvxpy as cp
import numpy as np
import tqdm
import scipy
import math
from scipy.special import xlogy
import time

In [2]:
def channel_capacity(n, m, P, sum_x=1):
    '''
    Boyd and Vandenberghe, Convex Optimization, exercise 4.57 page 207
    Capacity of a communication channel.

    We consider a communication channel, with input X(t)∈{1,..,n} and
    output Y(t)∈{1,...,m}, for t=1,2,... .The relation between the
    input and output is given statistically:
    p_(i,j) = ℙ(Y(t)=i|X(t)=j), i=1,..,m  j=1,...,n

    The matrix P ∈ ℝ^(m*n) is called the channel transition matrix, and
    the channel is called a discrete memoryless channel. Assuming X has a
    probability distribution denoted x ∈ ℝ^n, i.e.,
    x_j = ℙ(X=j), j=1,...,n

    The mutual information between X and Y is given by
    ∑(∑(x_j p_(i,j)log_2(p_(i,j)/∑(x_k p_(i,k)))))
    Then channel capacity C is given by
    C = sup I(X;Y).
    With a variable change of y = Px this becomes
    I(X;Y)=  c^T x - ∑(y_i log_2 y_i)
    where c_j = ∑(p_(i,j)log_2(p_(i,j)))
    '''

    # n is the number of different input values
    # m is the number of different output values
    if n*m == 0:
        print('The range of both input and output values must be greater than zero')
        return 'failed', np.nan, np.nan

    # x is probability distribution of the input signal X(t)
    x = cp.Variable(shape=n)

    # y is the probability distribution of the output signal Y(t)
    # P is the channel transition matrix
    y = P@x

    # I is the mutual information between x and y
    c = np.sum(np.array((xlogy(P, P) / math.log(2))), axis=0)
    I = c@x + cp.sum(cp.entr(y) / math.log(2))

    # Channel capacity maximised by maximising the mutual information
    obj = cp.Maximize(I)
    constraints = [cp.sum(x) == sum_x,x >= 0]

    # Form and solve problem
    prob = cp.Problem(obj,constraints)
    prob.solve()
    if prob.status=='optimal':
        return prob.status, prob.value, x.value
    else:
        return prob.status, np.nan, np.nan

In [621]:
def calculate_base_D(px, H_thres, k, S, N, Phi):
    # H_thres = np.linspace(S[i], S[j], k+1)
    # H_thres[0] = S[0]
    # H_thres[-1] = S[-1]
    
    Ax = np.zeros((k, N))
    for m in range(N):
        for n in range(k):
            Ax[n, m] = Phi[m].cdf(H_thres[n+1]) - Phi[m].cdf(H_thres[n])

    Ay = np.zeros((N, k))
    for m in range(N):
        for n in range(k):
            if np.round(np.sum(px*Ax[n,:]), 10) == 0:
                Ay[m, n] = 0
            else:
                Ay[m, n] = px[m]*Ax[n, m]/np.sum(px*Ax[n,:])

    py = np.matmul(Ax, px)
    c = np.sum(np.array((xlogy(Ay, Ay) / math.log(2))), axis=0)
    H_xy = -np.sum(py*c)
    
    return H_xy

In [4]:
def run_dp(px, N, M, K, S, Phi):
    D = np.zeros((M, M, K))
    H = np.zeros((K, M))

    for k in range(K):
        for j in tqdm.tqdm(range(M)):
            for i in range(j):
                tmp = []
                for q in range(j):
                    tmp.append(calculate_D(px, i, q, k-1, S, N, Phi) +\
                                           calculate_D(px, q+1, j, 1, S, N, Phi))
                D[i, j, k] = np.min(tmp)
                H[k, j] = np.argmin(tmp)
    H_K = M-1
    H_opt = []
    for i in np.arange(K-1)[::-1]:
        H_ = H[i+1, int(H_K)]
        H_opt.append(H_)
        H_K = H_

    H_opt = H_opt[::-1]
    
    return D, H, H_opt

### init

In [394]:
(end-start)/M

0.8

In [687]:
X = np.array([-3, 0, 3])
N = len(X)
M = 20
start = -8
end = 8
step = (end-start)/M
S = np.linspace(-8, 8, M+1)
K = 3

In [688]:
sigma = 0.1
U = X + np.random.randn(N)*sigma

In [689]:
Phi = [scipy.stats.norm(loc=X[i], scale=sigma) for i in range(N)]

In [690]:
Hx = [S[0], -1.5, 1.5, S[-1]]

A = np.zeros((K, N))

for j in range(N):
    for i in range(K):
        A[i, j] = Phi[j].cdf(Hx[i+1]) - Phi[j].cdf(Hx[i])

In [692]:
len(S)

21

In [297]:
channel_capacity(N, K, A)

('optimal', 1.584962500721156, array([0.33333333, 0.33333333, 0.33333333]))

### test

In [298]:
px = np.array([1.0/3, 1.0/3, 1.0/3])

In [699]:
i=5
j=6
k=1

H_thres = [S[i]]
s = int(((S[j]-S[i])/k)/step) + 1
for m in range(1, k):
    H_thres.append(S[i+m*s])
H_thres.append(S[j])

Ax = np.zeros((k, N))
for m in range(N):
    for n in range(k):
        Ax[n, m] = Phi[m].cdf(H_thres[n+1]) - Phi[m].cdf(H_thres[n])

Ay = np.zeros((N, k))
for m in range(N):
    for n in range(k):
        if np.round(np.sum(px*Ax[n,:]), 10) == 0:
            Ay[m, n] = 0
        else:
            Ay[m, n] = px[m]*Ax[n, m]/np.sum(px*Ax[n,:])

py = np.matmul(Ax, px)
c = np.sum(np.array((-xlogy(Ay, Ay) / math.log(2))), axis=0)
H_xy = np.sum(py*c)

In [684]:
Ax

array([[2.27501319e-002, 5.45208060e-225, 0.00000000e+000]])

In [685]:
py

array([0.00758338])

In [686]:
H_xy

1.3439906325146981e-222

In [598]:
H_thres = [S[i]]
s = int(((S[j]-S[i])/k)/step) + 1
for m in range(1, k):
    H_thres.append(S[i+m*s])
H_thres.append(S[j])

In [619]:
def get_partition_thres(i, j, k, step, S):
    H_thres = [S[i]]
    s = int(((S[j]-S[i])/(k))/step) + 1
    for m in range(1, (k)):
        H_thres.append(S[i+m*s])
    H_thres.append(S[j])
    return H_thres

In [620]:
get_partition_thres(1,1,1,step,S)

[-7.2, -7.2]

In [676]:
D = np.ones((M+1, M+1, K+1))*np.inf
H = np.ones((M+1, M+1, K+1))*np.nan

D[:,0,:] = 0
D[:,:,0] = 0

for k in range(1, K+1):
    for j in tqdm.tqdm(range(1, M+1)):
        for i in range(j):
            _, arg = calc_D(i,j,k, D)
            H[i,j,k] = arg

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 20.46it/s]


In [669]:
def calc_D(i, j, k, data):
    if data[i,j,k] == np.inf:
        # print(i,j,k)
        if k == 1 or j-i == 1:
            # print("base",i,j,k, calculate_base_D(px, [S[i], S[j]], 1, S, N, Phi))
            return calculate_base_D(px, [S[i], S[j]], 1, S, N, Phi), 0
        tmp = []
        for q in range(i+1, j):
            # print(q)
            D1, _ = calc_D(i, q, k-1, data)
            D2, _ = calc_D(q+1, j, 1, data)
            tmp.append(D1 + D2)
        data[i,j,k] = min(tmp)
        return min(tmp), np.argmin(tmp)
    return data[i,j,k], 0

In [655]:
D = np.ones((M+1, M+1, K+1))*np.inf

In [693]:
K

3

In [704]:
P = []
for i in range(len(S)-1):
    P.append(calculate_base_D(px, [S[i], S[i+1]], 1, S, N, Phi))

In [695]:
M = np.zeros((len(S)-1, 3))

In [702]:
M[0,:] = calculate_base_D(px, [S[0], S[1]], 1, S, N, Phi)
M[:,0] = calculate_base_D(px, [S[0], S[-1]], 1, S, N, Phi)

In [703]:
for i in range(1, len(S)-1):
    for j in range(1, K):
        M[i,k] = np.inf
        for q in range(i-1):
            s = np.min(M[q, j-1], )

array([[ 1.5849625, -0.       , -0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ],
       [ 1.5849625,  0.       ,  0.       ]])

In [560]:
calculate_D(px, 5, 15, 2, S, N, Phi)

0.666666666666678

In [561]:
calculate_D(px, 5, 15, 1, S, N, Phi)

1.584962500721156

In [617]:
thres

[7.200000000000001, 8.0, 7.200000000000001]

In [618]:
get_partition_thres(i, q, k-1, step)

IndexError: index 21 is out of bounds for axis 0 with size 21

In [272]:
H_K1 = M
H_opt = []
for i in np.arange(1, K)[::-1]:
    H_ = H[i+1, int(H_K1)]
    H_opt.append(H_)
    H_K1 = H_

H_opt = H_opt[::-1]

In [278]:
D[1, M, K]

1.584962500721156

In [280]:
D[1,M,:]

array([0.       , 1.5849625, 1.5849625, 1.5849625])

In [228]:
[S[int(l)] for l in H_opt]

ValueError: cannot convert float NaN to integer

In [307]:
S

array([-8. , -7.2, -6.4, -5.6, -4.8, -4. , -3.2, -2.4, -1.6, -0.8,  0. ,
        0.8,  1.6,  2.4,  3.2,  4. ,  4.8,  5.6,  6.4,  7.2,  8. ])

## alternating

In [60]:
I_list = []
px_list = []
h_list = []

for iteration in tqdm.tqdm(range(100)):
    start = time.time()
    # step 1
    status, obj_value, px_value = channel_capacity(N, K, A)

    # step 2
    D = np.zeros((N, M, K))
    H = np.zeros((K, M))

    for k in range(1, K):
        for i in range(N):
            for j in range(M):
                tmp = []
                for q in range(j):
                    tmp.append(calculate_D(px_value, i, q, k-1, S, N, Phi) +\
                                calculate_D(px_value, q+1, j, 1, S, N, Phi))
                if len(tmp) > 0:
                    D[i, j, k] = np.min(tmp)
                    H[k, j] = np.argmin(tmp)
                else:
                    D[i, j, k] = 0
                    H[k, j] = 0

    H_K = M-1
    H_opt = []
    for i in np.arange(K-1)[::-1]:
        H_ = H[i+1, int(H_K)]
        H_opt.append(H_)
        H_K = H_

    H_opt = H_opt[::-1]

    # recompute A
    thres = [S[int(l)] for l in H_opt]
    Hx = [S[0]] + thres + [S[-1]]

    A = np.zeros((K, N))

    for j in range(N):
        for i in range(K):
            A[i, j] = Phi[j].cdf(Hx[i+1]) - Phi[j].cdf(Hx[i])

    # calculate I(X;Y)
    py = np.matmul(A, px_value)

    # I is the mutual information between x and y
    c = np.sum(np.array((xlogy(A, A) / math.log(2))), axis=0)
    I = -np.sum(py*c) - np.sum(np.array((xlogy(py, py) / math.log(2))), axis=0)
    stop = time.time()
    print("iter {} - I(X;Y)={:.6f} - took {:.2f}s".format(iteration, I, stop-start))
    print("threshold = ", thres)
    print("H_opt = ", H_opt)
    print("px = ", px_value)
    
    I_list.append(I)
    h_list.append(H_opt)
    px_list.append(px_value)

  1%|▍                                       | 1/100 [05:21<8:50:17, 321.39s/it]

iter 0 - I(X;Y)=2.981629 - took 321.39s
threshold =  [-0.6999999999999993, -0.5999999999999996, 4.0]
H_opt =  [73.0, 74.0, 120.0]
px =  [4.79280467e-01 6.40236081e-10 4.92278394e-10 5.20719532e-01]


  2%|▊                                       | 2/100 [10:41<8:43:32, 320.54s/it]

iter 1 - I(X;Y)=2.988892 - took 319.94s
threshold =  [-0.5999999999999996, -0.5, 4.100000000000001]
H_opt =  [74.0, 75.0, 121.0]
px =  [4.97742142e-01 2.09813968e-08 2.10434996e-08 5.02257816e-01]


  3%|█▏                                      | 3/100 [15:48<8:28:27, 314.51s/it]

iter 2 - I(X;Y)=2.988954 - took 307.34s
threshold =  [-0.5999999999999996, -0.5, 4.100000000000001]
H_opt =  [74.0, 75.0, 121.0]
px =  [4.98369779e-01 1.76745245e-08 1.77115076e-08 5.01630186e-01]


  3%|█▏                                      | 3/100 [18:10<9:47:43, 363.54s/it]


KeyboardInterrupt: 